In [13]:
!pip install 'descarteslabs[complete]'

In [16]:
!python -V

Python 3.7.1


In [1]:
import os
import descarteslabs as dl 
from descarteslabs.client.services.tasks import AsyncTasks, as_completed
from descarteslabs.client.services.catalog import Catalog

/Users/jtc/.pyenv/versions/3.7.2/Python.framework/Versions/3.7/lib/python3.7/site-packages/geojson/mapping.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping


In [ ]:
def rice_sar_stats(year, product_id, tile):

    import json
    import numpy as np
    import logging
    import time
    from datetime import datetime
    catalog = Catalog()

    def calculate_stats(arr):

        vx_max = arr.max(axis=0)
        vx_min = arr.min(axis=0)
        vx_mean = arr.mean(axis=0)
        vx_med = np.ma.median(arr, axis=0)
        vx_std = arr.std(axis=0)

        stats = [vx_max, vx_min, vx_mean, vx_med, vx_std]

        return stats
    
    def get_imagery(year, tile, s1_pass, band):
        
        start = str(year) + '-01-01'
        end = str(year) + '-12-31'
        
        dlkey = tile['properties']['key']
        
        scenes, geoctx = dl.scenes.search(
            tile['geometry'],
            products=["sentinel-1:GRD"],
            start_datetime=start,
            end_datetime=end,
            limit=1000
        )
        
        # scenes['features'] DNE...
#         if s1_pass in ['DESCENDING', 'ASCENDING']:
#             scenes['features'] = [
#                 ft for ft in scenes['features'] if ft['properties']['pass'] == s1_pass
#             ]

        geoctx = geoctx.assign(resolution=20)
        stack = scenes.stack(
            band,
            geoctx,
            mask_alpha=False
        )
        
        # https://docs.descarteslabs.com/descarteslabs/scenes/docs/scenecollection.html
        # returned array’s shape is (scene, band, y, x) if bands_axis is 1
        # only have 1 band, so take the first
        _stack = stack[:,0,:,:]
        return _stack, geoctx

    def catalog_upload(tile, stats_list, meta, year):
        # Upload the ndarray as a single scene in our new product
        t_props = tile['properties']
        
        catalog.upload_ndarray(
            stats_list,
            product_id=product_id,
            image_id=t_props['key'],
            proj4=t_props['proj4'],
            geotrans=t_props['geotrans']
        )
        return

    def run_analysis(year, tile):

        bands = [['vv'], ['vh']]
        passes = ['DESCENDING', 'ASCENDING', 'BOTH']
        all_stats = []

        for _ib, band in enumerate(bands):
            print(f"\tBAND {_ib + 1}/{len(bands)}: {band}")
            for _ip, s1_pass in enumerate(passes):
                print(f"\t\tPASS {_ip + 1}/{len(passes)}: {s1_pass}")
                
                try:
                    imagery, meta = get_imagery(year, tile, s1_pass, band)
                    arr = np.ma.masked_equal(imagery, 0)
                    stats = calculate_stats(arr)
                    all_stats.append(stats)
                except Exception as e:
                    print(e)
                    tile_size = tile['properties']['tilesize']
                    nodata = np.zeros((tile_size, tile_size), dtype=np.uint8)
                    all_stats.append([nodata, nodata, nodata, nodata, nodata])

#         # ---------------------
#         # TEST IT FASTER WITH ONE BAND
#         band = ['vv']
#         s1_pass = 'DESCENDING'
#         imagery, meta = get_imagery(year, tile, s1_pass, band)
#         print(meta)
#         arr = np.ma.masked_equal(imagery, 0)
#         stats = calculate_stats(arr)
#         all_stats.append(stats)
#         # ---------------------
         
        # flatten the list of lists (all_stats) into a single list
        stats_list = [item for sublist in all_stats for item in sublist]
        stats_ndarray = np.array(stats_list)
        
        # re-shape stats_ndarray as (x, y, band (?))
        stats_ndarray = np.transpose(stats_ndarray, (1,2,0))
        catalog_upload(tile, stats_ndarray, meta, year)
        return
    
    return run_analysis(year,tile)



In [82]:
""" For testing
"""
year = '2017'
loc = 'singapore'
    
res = 20.0
pad = 0
pixels = 1024

aoi = dl.places.shape(loc, geom = 'low')
tiles = dl.raster.dltiles_from_shape(res, pixels, pad, aoi['geometry'])

# get username to generate product_id
user = os.environ.get('USER')

product_id = '{}:{}:test'.format(
    dl.Auth().namespace,
    ''.join(user.lower().split(' '))
)

for _it, tile_ft in enumerate(tiles['features']):

    print(f"\nTILE {_it + 1}/{len(tiles['features'])}\n")
    
    rice_sar_stats(
        year,
        product_id,
        tile_ft
    )

band ['vv'], pass DESCENDING
stack.shape: (78, 1, 1024, 1026)
_stack.shape: (78, 1024, 1026)
AOI(geometry=<shapely.geom...t 0x11883bf98>,
    resolution=20,
    crs='EPSG:32648',
    align_pixels=True,
    bounds=(103.52739979787773, 1.11136300154557, 103.71152831939709, 1.296690579216693),
    bounds_crs='EPSG:4326',
    shape=None)
band ['vv'], pass ASCENDING
stack.shape: (78, 1, 1024, 1026)
_stack.shape: (78, 1024, 1026)
AOI(geometry=<shapely.geom...t 0x118b652b0>,
    resolution=20,
    crs='EPSG:32648',
    align_pixels=True,
    bounds=(103.52739979787773, 1.11136300154557, 103.71152831939709, 1.296690579216693),
    bounds_crs='EPSG:4326',
    shape=None)
band ['vv'], pass BOTH
stack.shape: (78, 1, 1024, 1026)
_stack.shape: (78, 1024, 1026)
AOI(geometry=<shapely.geom...t 0x118b65320>,
    resolution=20,
    crs='EPSG:32648',
    align_pixels=True,
    bounds=(103.52739979787773, 1.11136300154557, 103.71152831939709, 1.296690579216693),
    bounds_crs='EPSG:4326',
    shape=None)

stack.shape: (55, 1, 1024, 1026)
_stack.shape: (55, 1024, 1026)
AOI(geometry=<shapely.geom...t 0x116d9b7f0>,
    resolution=20,
    crs='EPSG:32648',
    align_pixels=True,
    bounds=(103.5272850902188, 1.29658955979125, 103.71144133199714, 1.481931293654704),
    bounds_crs='EPSG:4326',
    shape=None)
band ['vh'], pass DESCENDING
stack.shape: (55, 1, 1024, 1026)
_stack.shape: (55, 1024, 1026)
AOI(geometry=<shapely.geom...t 0x116da11d0>,
    resolution=20,
    crs='EPSG:32648',
    align_pixels=True,
    bounds=(103.5272850902188, 1.29658955979125, 103.71144133199714, 1.481931293654704),
    bounds_crs='EPSG:4326',
    shape=None)
band ['vh'], pass ASCENDING
stack.shape: (55, 1, 1024, 1026)
_stack.shape: (55, 1024, 1026)
AOI(geometry=<shapely.geom...t 0x117a862b0>,
    resolution=20,
    crs='EPSG:32648',
    align_pixels=True,
    bounds=(103.5272850902188, 1.29658955979125, 103.71144133199714, 1.481931293654704),
    bounds_crs='EPSG:4326',
    shape=None)
band ['vh'], pass BOTH
st

stack.shape: (78, 1, 1024, 1026)
_stack.shape: (78, 1024, 1026)
AOI(geometry=<shapely.geom...t 0x116d046d8>,
    resolution=20,
    crs='EPSG:32648',
    align_pixels=True,
    bounds=(103.8954962607725, 1.111524634720008, 104.07962481867163, 1.296852237440217),
    bounds_crs='EPSG:4326',
    shape=None)
band ['vv'], pass BOTH
stack.shape: (78, 1, 1024, 1026)
_stack.shape: (78, 1024, 1026)
AOI(geometry=<shapely.geom...t 0x118bc55f8>,
    resolution=20,
    crs='EPSG:32648',
    align_pixels=True,
    bounds=(103.8954962607725, 1.111524634720008, 104.07962481867163, 1.296852237440217),
    bounds_crs='EPSG:4326',
    shape=None)
band ['vh'], pass DESCENDING
stack.shape: (78, 1, 1024, 1026)
_stack.shape: (78, 1024, 1026)
AOI(geometry=<shapely.geom...t 0x11895d630>,
    resolution=20,
    crs='EPSG:32648',
    align_pixels=True,
    bounds=(103.8954962607725, 1.111524634720008, 104.07962481867163, 1.296852237440217),
    bounds_crs='EPSG:4326',
    shape=None)
band ['vh'], pass ASCENDING

ValueError: could not broadcast input array from shape (1024,1026) into shape (1024)

In [24]:
year = '2017'
loc = 'singapore'
    
res = 20.0
pad = 0
pixels = 1024 

aoi = dl.places.shape(loc, geom = 'low')
tiles = dl.raster.dltiles_from_shape(res, pixels, pad, aoi['geometry'])

# get username to generate product_id
user = os.environ.get('USER')
    
product_id = '{}:{}:test'.format(
    dl.Auth().namespace,
    ''.join(user.lower().split(' '))
)


try:
    at = AsyncTasks()
    async_function = at.create_function(
            rice_sar_stats, cpus=1, memory="6Gi", name="sar_img_stats",
            maximum_concurrency = 14,
            image='us.gcr.io/dl-ci-cd/images/tasks/public/py3.7/default:v2018.11.27')
    # Iterate over tiles, submit tasks
#     tasks = []
#     for tile in tiles['features']:

#         print("TILE ID: %s" % tile['properties']['key'])

#         t = async_function( 
#             year,
#             product_id,
#             tile,
#         )
#         tasks.append(t)

    # make it quick => only one tile
    tasks = [async_function(
        year,
        product_id,
        tiles['features'][0]
    )]

    print('Done submitting %i tasks. Starting to collect' % len(tasks))

    for task in as_completed(tasks):

        if task.exception:
            print("Failed with exception: ", task.log)
        else:
            task.result
except:
    print ("tasks not submitted!")
    

Done submitting 1 tasks. Starting to collect


Failed with exception:  b'band vv, pass DESCENDING\nBand \'alpha\' is not available in the product \'sentinel-1:GRD\'\nband vv, pass ASCENDING\nBand \'alpha\' is not available in the product \'sentinel-1:GRD\'\nband vv, pass BOTH\nBand \'alpha\' is not available in the product \'sentinel-1:GRD\'\nband vh, pass DESCENDING\nBand \'alpha\' is not available in the product \'sentinel-1:GRD\'\nband vh, pass ASCENDING\nBand \'alpha\' is not available in the product \'sentinel-1:GRD\'\nband vh, pass BOTH\nBand \'alpha\' is not available in the product \'sentinel-1:GRD\'\nTraceback (most recent call last):\n  File "/opt/src/wrapper/wrapper.py", line 578, in pickle_work\n    output = f(*arguments, **parameters)\n  File "<ipython-input-23-047c1b5f3393>", line 127, in rice_sar_stats\n  File "<ipython-input-23-047c1b5f3393>", line 124, in run_analysis\nUnboundLocalError: local variable \'meta\' referenced before assignment\n'
